# Main

In [133]:
import pandas as pd
import numpy as np
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score
from sklearn.ensemble import VotingClassifier
from sklearn.pipeline import make_pipeline
from collections import Counter
import matplotlib.pyplot as plt

In [134]:
dataset=pd.read_csv("./Cardiovascular_Disease_Dataset.csv")

In [135]:
dataset.shape

(1000, 14)

In [136]:
from sklearn.model_selection import train_test_split

# Assuming X is your feature matrix and y is your target variable
X = dataset.drop(columns=['target'])
y = dataset['target']

# Split the data into training and testing sets
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=32)

# Split the training data into training and validation sets
X_train, X_val, y_train, y_val = train_test_split(X_train, y_train, test_size=0.2, random_state=42)

# Print the shapes of the training, validation, and testing sets
print("Shape of X_train:", X_train.shape)
print("Shape of X_val:", X_val.shape)
print("Shape of X_test:", X_test.shape)
print("Shape of y_train:", y_train.shape)
print("Shape of y_val:", y_val.shape)
print("Shape of y_test:", y_test.shape)


Shape of X_train: (640, 13)
Shape of X_val: (160, 13)
Shape of X_test: (200, 13)
Shape of y_train: (640,)
Shape of y_val: (160,)
Shape of y_test: (200,)


In [137]:
from sklearn.ensemble import (
    StackingClassifier,    RandomForestClassifier,     GradientBoostingClassifier,     AdaBoostClassifier,     ExtraTreesClassifier)
from sklearn.tree import DecisionTreeClassifier
from sklearn.naive_bayes import GaussianNB
from sklearn.linear_model import LogisticRegression
from sklearn.neighbors import KNeighborsClassifier
from sklearn.svm import SVC
import xgboost as xgb

# Base models
rf = RandomForestClassifier(criterion='entropy', max_features='sqrt')
dt = DecisionTreeClassifier(criterion='entropy', max_depth=5, min_samples_leaf=5, splitter='random')
gb = GradientBoostingClassifier(max_depth=4, min_samples_leaf=20)
gnb = GaussianNB(var_smoothing=0.004328761281083057)
svm = SVC(C=100, gamma='auto', probability=True)
knn = KNeighborsClassifier(metric='manhattan', n_neighbors=9, weights='distance')
lr = LogisticRegression(C=0.615848211066026, max_iter=500, penalty='l1', solver='liblinear')

# Additional models
et = ExtraTreesClassifier(n_estimators=100, criterion='entropy', max_features='sqrt')
adb = AdaBoostClassifier(n_estimators=50, learning_rate=1.0)
xgb_clf = xgb.XGBClassifier(max_depth=4, learning_rate=0.1, n_estimators=100, use_label_encoder=False, eval_metric='logloss')

# Creating a list of all classifiers
models = {
    "Random Forest": rf,    "Decision Tree": dt,    "Gradient Boosting": gb,    "GaussianNB": gnb,    "SVM": svm,    "KNN": knn,    "Logistic Regression": lr,    "Extra Trees": et,    "AdaBoost": adb,    "XGBoost": xgb_clf}


In [138]:
es1 = [('clf1', et), ('clf2', rf), ('clf3', gb), ('clf4', dt)]
es2 = [('clf1', et), ('clf2', rf), ('clf3', gb), ('clf4', dt)]
es3 = [('clf1', et), ('clf2', rf), ('clf3', gb), ('clf4', dt)]
es4 = [('clf1', et), ('clf2', rf), ('clf3', gb), ('clf4', dt)]  #gnb replaced with dt

ev1 = [('clf1', rf), ('clf2', dt), ('clf3', gb), ('clf4', gnb)]
ev2 = [('clf1', dt), ('clf2', gb), ('clf3', svm), ('clf4', adb)]
ev3 = [('clf1', rf), ('clf2', knn), ('clf3', dt), ('clf4', gb)]
ev4 = [('clf1', dt), ('clf2', gb), ('clf3', lr), ('clf4', svm)]
ev5 = []


es5 = []

# <h2 style="Color:Red"> Stack 1

Define Majority voting

In [139]:
# Create a VotingClassifier with base classifiers clf1, clf2, clf3, and clf4
voting_clf1 = VotingClassifier(
    estimators=ev1,
    voting='soft'
)
# Create a StackingClassifier with base classifiers clf1, clf2, clf3, and clf4, and meta-classifier voting_clf
sclf1 = StackingClassifier(
    estimators=es1,
    final_estimator=voting_clf1
)

# Fit the Stacking Classifier to the training data
sclf1.fit(X_train, y_train)

# Create a pipeline with the Stacking Classifier and the final logistic regression model
pipeline = make_pipeline(sclf1, voting_clf1)

# Use the pipeline to transform the training data into the feature space of the logistic regression model
transformed_X_train1 = pipeline[:-1].transform(X_train)

# Now you can inspect the transformed training data
# For example, you can print the first few rows
# print(transformed_X_train1[:5])  # Print the first 5 rows

# Optionally, you can perform further analysis or visualization on this transformed data
# For example, you can plot histograms or scatter plots to explore the distribution of features

from sklearn.metrics import confusion_matrix
import seaborn as sns
def calculate_false_negative(y_true, y_pred):
    cm = confusion_matrix(y_true, y_pred)
    tn, fp, fn, tp = cm.ravel()
    return fn / (fn + tp) * 100

from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score
from sklearn.metrics import confusion_matrix
import seaborn as sns
import matplotlib.pyplot as plt


# Define the function to calculate false negative percentage


# Assuming you have defined the necessary variables and models

print("Accuracy for stack 1(Validation): ")

# Make predictions on the validation set
val_predictions1 = sclf1.predict(X_val)

# Calculate accuracy
accuracy = accuracy_score(y_val, val_predictions1)
print("Validation Accuracy: %0.4f" % accuracy)

# Calculate precision
precision = precision_score(y_val, val_predictions1, average='weighted')
print("Validation Precision: %0.4f" % precision)

# Calculate recall
recall = recall_score(y_val, val_predictions1, average='weighted')
print("Validation Recall: %0.4f" % recall)

# Calculate F1 score
f1 = f1_score(y_val, val_predictions1, average='weighted')
print("Validation F1 Score: %0.4f" % f1)

# Plot confusion matrix for validation data
# cm = confusion_matrix(y_val, val_predictions1)
# plt.figure(figsize=(8, 6))
# sns.heatmap(cm, annot=True, fmt="d", cmap="Blues", xticklabels=np.unique(y_val), yticklabels=np.unique(y_val))
# plt.title("Confusion Matrix - Validation Data")
# plt.xlabel('Predicted')
# plt.ylabel('Actual')
# plt.show()

# Calculate false negative percentage for validation data
false_negative_percentage = calculate_false_negative(y_val, val_predictions1)
print(f"False Negative Percentage for Validation Data: {false_negative_percentage:.2f}%")

print()
print()

print("Accuracy for stack 1(Test): ")

# Make predictions on the test set
test_predictions1 = sclf1.predict(X_test)

# Calculate accuracy
test_accuracy = accuracy_score(y_test, test_predictions1)
print("Test Accuracy: %0.4f" % test_accuracy)

# Calculate precision
test_precision = precision_score(y_test, test_predictions1, average='weighted')
print("Test Precision: %0.4f" % test_precision)

# Calculate recall
test_recall = recall_score(y_test, test_predictions1, average='weighted')
print("Test Recall: %0.4f" % test_recall)

# Calculate F1 score
test_f1 = f1_score(y_test, test_predictions1, average='weighted')
print("Test F1 Score: %0.4f" % test_f1)

# Plot confusion matrix for test data
# cm = confusion_matrix(y_test, test_predictions1)
# plt.figure(figsize=(8, 6))
# sns.heatmap(cm, annot=True, fmt="d", cmap="Blues", xticklabels=np.unique(y_test), yticklabels=np.unique(y_test))
# plt.title("Confusion Matrix - Test Data")
# plt.xlabel('Predicted')
# plt.ylabel('Actual')
# plt.show()

# Calculate false negative percentage for test data
false_negative_percentage = calculate_false_negative(y_test, test_predictions1)
print(f"False Negative Percentage for Test Data: {false_negative_percentage:.2f}%")


Accuracy for stack 1(Validation): 
Validation Accuracy: 0.9750
Validation Precision: 0.9752
Validation Recall: 0.9750
Validation F1 Score: 0.9749
False Negative Percentage for Validation Data: 1.01%


Accuracy for stack 1(Test): 
Test Accuracy: 0.9900
Test Precision: 0.9900
Test Recall: 0.9900
Test F1 Score: 0.9900
False Negative Percentage for Test Data: 0.93%


# <h2 style="Color:Red"> Stack 2

In [140]:
# Create a VotingClassifier with base classifiers clf1, clf2, clf3, and clf4
voting_clf2 = VotingClassifier(
    estimators=ev2,
    voting='soft'
)
# Create a StackingClassifier with base classifiers clf1, clf2, clf3, and clf4, and meta-classifier voting_clf
sclf2 = StackingClassifier(
    estimators=es2,
    final_estimator=voting_clf2
)



# Fit the Stacking Classifier to the training data
sclf2.fit(X_train, y_train)

# Create a pipeline with the Stacking Classifier and the final logistic regression model
pipeline = make_pipeline(sclf2, voting_clf2)

# Use the pipeline to transform the training data into the feature space of the logistic regression model
transformed_X_train2 = pipeline[:-1].transform(X_train)


# print(transformed_X_train2[:5])  # Print the first 5 rows

print("Accuracy for stack 2(Validation): ")

# Make predictions on the validation set
val_predictions2 = sclf2.predict(X_val)

# Calculate accuracy
accuracy = accuracy_score(y_val, val_predictions2)
print("Validation Accuracy: %0.4f" % accuracy)

# Calculate precision
precision = precision_score(y_val, val_predictions2, average='weighted')
print("Validation Precision: %0.4f" % precision)

# Calculate recall
recall = recall_score(y_val, val_predictions2, average='weighted')
print("Validation Recall: %0.4f" % recall)

# Calculate F1 score
f1 = f1_score(y_val, val_predictions2, average='weighted')
print("Validation F1 Score: %0.4f" % f1)

# Plot confusion matrix for validation data
# cm = confusion_matrix(y_val, val_predictions2)
# plt.figure(figsize=(8, 6))
# sns.heatmap(cm, annot=True, fmt="d", cmap="Blues", xticklabels=np.unique(y_val), yticklabels=np.unique(y_val))
# plt.title("Confusion Matrix - Validation Data")
# plt.xlabel('Predicted')
# plt.ylabel('Actual')
# plt.show()

# Calculate false negative percentage for validation data
false_negative_percentage = calculate_false_negative(y_val, val_predictions2)
print(f"False Negative Percentage for Validation Data: {false_negative_percentage:.2f}%")


print()
print()

print("Accuracy for stack 2(Test): ")

# Make predictions on the test set
test_predictions2 = sclf2.predict(X_test)

# Calculate accuracy
test_accuracy = accuracy_score(y_test, test_predictions2)
print("Test Accuracy: %0.4f" % test_accuracy)

# Calculate precision
test_precision = precision_score(y_test, test_predictions2, average='weighted')
print("Test Precision: %0.4f" % test_precision)

# Calculate recall
test_recall = recall_score(y_test, test_predictions2, average='weighted')
print("Test Recall: %0.4f" % test_recall)

# Calculate F1 score
test_f1 = f1_score(y_test, test_predictions2, average='weighted')
print("Test F1 Score: %0.4f" % test_f1)

# Plot confusion matrix for test data
# cm = confusion_matrix(y_test, test_predictions2)
# plt.figure(figsize=(8, 6))
# sns.heatmap(cm, annot=True, fmt="d", cmap="Blues", xticklabels=np.unique(y_test), yticklabels=np.unique(y_test))
# plt.title("Confusion Matrix - Test Data")
# plt.xlabel('Predicted')
# plt.ylabel('Actual')
# plt.show()

# Calculate false negative percentage for test data
false_negative_percentage = calculate_false_negative(y_test, test_predictions2)
print(f"False Negative Percentage for Test Data: {false_negative_percentage:.2f}%")



Accuracy for stack 2(Validation): 
Validation Accuracy: 0.9750
Validation Precision: 0.9752
Validation Recall: 0.9750
Validation F1 Score: 0.9749
False Negative Percentage for Validation Data: 1.01%


Accuracy for stack 2(Test): 
Test Accuracy: 0.9700
Test Precision: 0.9707
Test Recall: 0.9700
Test F1 Score: 0.9699
False Negative Percentage for Test Data: 0.93%


c:\Users\user\AppData\Local\Programs\Python\Python39\lib\site-packages\sklearn\ensemble\_weight_boosting.py:519: FutureWarning: The SAMME.R algorithm (the default) is deprecated and will be removed in 1.6. Use the SAMME algorithm to circumvent this warning.
  warnings.warn(


# <h2 style="Color:Red"> Stack 3

In [141]:
voting_clf3 = VotingClassifier(
    estimators=ev3,
    voting='soft'
)

# Create a StackingClassifier with base classifiers clf1, clf2, clf3, and clf4, and meta-classifier voting_clf
sclf3 = StackingClassifier(
    estimators=es3,
    final_estimator=voting_clf3
)


# Fit the Stacking Classifier to the training data
sclf3.fit(X_train, y_train)

# Create a pipeline with the Stacking Classifier and the final logistic regression model
pipeline = make_pipeline(sclf3, voting_clf3)

# Use the pipeline to transform the training data into the feature space of the logistic regression model
transformed_X_train3 = pipeline[:-1].transform(X_train)


# print(transformed_X_train3[:5])  # Print the first 5 rows

print("Accuracy for stack 3(Validation): ")

# Make predictions on the validation set
val_predictions3 = sclf3.predict(X_val)

# Calculate accuracy
accuracy = accuracy_score(y_val, val_predictions3)
print("Validation Accuracy: %0.4f" % accuracy)

# Calculate precision
precision = precision_score(y_val, val_predictions3, average='weighted')
print("Validation Precision: %0.4f" % precision)

# Calculate recall
recall = recall_score(y_val, val_predictions3, average='weighted')
print("Validation Recall: %0.4f" % recall)

# Calculate F1 score
f1 = f1_score(y_val, val_predictions3, average='weighted')
print("Validation F1 Score: %0.4f" % f1)

# # Plot confusion matrix for validation data
# cm = confusion_matrix(y_val, val_predictions3)
# plt.figure(figsize=(8, 6))
# sns.heatmap(cm, annot=True, fmt="d", cmap="Blues", xticklabels=np.unique(y_val), yticklabels=np.unique(y_val))
# plt.title("Confusion Matrix - Validation Data")
# plt.xlabel('Predicted')
# plt.ylabel('Actual')
# plt.show()

# Calculate false negative percentage for validation data
false_negative_percentage = calculate_false_negative(y_val, val_predictions3)
print(f"False Negative Percentage for Validation Data: {false_negative_percentage:.2f}%")


print()
print()

print("Accuracy for stack 3(Test): ")

# Make predictions on the test set
test_predictions3 = sclf3.predict(X_test)

# Calculate accuracy
test_accuracy = accuracy_score(y_test, test_predictions3)
print("Test Accuracy: %0.4f" % test_accuracy)

# Calculate precision
test_precision = precision_score(y_test, test_predictions3, average='weighted')
print("Test Precision: %0.4f" % test_precision)

# Calculate recall
test_recall = recall_score(y_test, test_predictions3, average='weighted')
print("Test Recall: %0.4f" % test_recall)

# Calculate F1 score
test_f1 = f1_score(y_test, test_predictions3, average='weighted')
print("Test F1 Score: %0.4f" % test_f1)

# Plot confusion matrix for test data
# cm = confusion_matrix(y_test, test_predictions3)
# plt.figure(figsize=(8, 6))
# sns.heatmap(cm, annot=True, fmt="d", cmap="Blues", xticklabels=np.unique(y_test), yticklabels=np.unique(y_test))
# plt.title("Confusion Matrix - Test Data")
# plt.xlabel('Predicted')
# plt.ylabel('Actual')
# plt.show()

# Calculate false negative percentage for test data
false_negative_percentage = calculate_false_negative(y_test, test_predictions3)
print(f"False Negative Percentage for Test Data: {false_negative_percentage:.2f}%")



Accuracy for stack 3(Validation): 
Validation Accuracy: 0.9812
Validation Precision: 0.9818
Validation Recall: 0.9812
Validation F1 Score: 0.9812
False Negative Percentage for Validation Data: 0.00%


Accuracy for stack 3(Test): 
Test Accuracy: 0.9800
Test Precision: 0.9807
Test Recall: 0.9800
Test F1 Score: 0.9800
False Negative Percentage for Test Data: 0.00%


# <h2 style="Color:Red"> Stack 4

In [142]:
voting_clf4 = VotingClassifier(
    estimators=ev4,
    voting='soft'
)


# Create a StackingClassifier with base classifiers clf1, clf2, clf3, and clf4, and meta-classifier voting_clf
sclf4 = StackingClassifier(
    estimators=ev4,
    final_estimator=voting_clf4
    
)
# Fit the Stacking Classifier to the training data
sclf4.fit(X_train, y_train)

# Create a pipeline with the Stacking Classifier and the final logistic regression model
pipeline = make_pipeline(sclf4, voting_clf4)

# Use the pipeline to transform the training data into the feature space of the logistic regression model
transformed_X_train4 = pipeline[:-1].transform(X_train)


# print(transformed_X_train4[:5])  # Print the first 5 rows

print("Accuracy for stack 4(Validation): ")

# Make predictions on the validation set
val_predictions4 = sclf4.predict(X_val)

# Calculate accuracy
accuracy = accuracy_score(y_val, val_predictions4)
print("Validation Accuracy: %0.4f" % accuracy)

# Calculate precision
precision = precision_score(y_val, val_predictions4, average='weighted')
print("Validation Precision: %0.4f" % precision)

# Calculate recall
recall = recall_score(y_val, val_predictions4, average='weighted')
print("Validation Recall: %0.4f" % recall)

# Calculate F1 score
f1 = f1_score(y_val, val_predictions4, average='weighted')
print("Validation F1 Score: %0.4f" % f1)

# Plot confusion matrix for validation data
# cm = confusion_matrix(y_val, val_predictions4)
# plt.figure(figsize=(8, 6))
# sns.heatmap(cm, annot=True, fmt="d", cmap="Blues", xticklabels=np.unique(y_val), yticklabels=np.unique(y_val))
# plt.title("Confusion Matrix - Validation Data")
# plt.xlabel('Predicted')
# plt.ylabel('Actual')
# plt.show()

# Calculate false negative percentage for validation data
false_negative_percentage = calculate_false_negative(y_val, val_predictions4)
print(f"False Negative Percentage for Validation Data: {false_negative_percentage:.2f}%")


print()
print()

print("Accuracy for stack 4(Test): ")

# Make predictions on the test set
test_predictions4 = sclf4.predict(X_test)

# Calculate accuracy
test_accuracy = accuracy_score(y_test, test_predictions4)
print("Test Accuracy: %0.4f" % test_accuracy)

# Calculate precision
test_precision = precision_score(y_test, test_predictions4, average='weighted')
print("Test Precision: %0.4f" % test_precision)

# Calculate recall
test_recall = recall_score(y_test, test_predictions4, average='weighted')
print("Test Recall: %0.4f" % test_recall)

# Calculate F1 score
test_f1 = f1_score(y_test, test_predictions4, average='weighted')
print("Test F1 Score: %0.4f" % test_f1)

# Plot confusion matrix for test data
# cm = confusion_matrix(y_test, test_predictions4)
# plt.figure(figsize=(8, 6))
# sns.heatmap(cm, annot=True, fmt="d", cmap="Blues", xticklabels=np.unique(y_test), yticklabels=np.unique(y_test))
# plt.title("Confusion Matrix - Test Data")
# plt.xlabel('Predicted')
# plt.ylabel('Actual')
# plt.show()

# Calculate false negative percentage for test data
false_negative_percentage = calculate_false_negative(y_test, test_predictions4)
print(f"False Negative Percentage for Test Data: {false_negative_percentage:.2f}%")



Accuracy for stack 4(Validation): 
Validation Accuracy: 0.9750
Validation Precision: 0.9752
Validation Recall: 0.9750
Validation F1 Score: 0.9749
False Negative Percentage for Validation Data: 1.01%


Accuracy for stack 4(Test): 
Test Accuracy: 0.9800
Test Precision: 0.9807
Test Recall: 0.9800
Test F1 Score: 0.9800
False Negative Percentage for Test Data: 0.00%


# <h1 style="Color: Violet"> Combined Majority voting

In [143]:
from collections import Counter
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score, confusion_matrix

# Define the function to calculate false negative percentage
def calculate_false_negative(y_true, y_pred):
    cm = confusion_matrix(y_true, y_pred)
    tn, fp, fn, tp = cm.ravel()
    return fn / (fn + tp) * 100

# Combine predictions using majority voting
combined_val_predictions = []
for p1, p2, p3, p4 in zip(val_predictions1, val_predictions2, val_predictions3, val_predictions4):
    votes = Counter([p1, p2, p3, p4])
    combined_val_predictions.append(votes.most_common(1)[0][0])

# Plot confusion matrix for the combined model
# cm_combined = confusion_matrix(y_val, combined_val_predictions)
# plt.figure(figsize=(8, 6))
# sns.heatmap(cm_combined, annot=True, fmt="d", cmap="Blues", xticklabels=np.unique(y_val), yticklabels=np.unique(y_val))
# plt.title("Confusion Matrix - Combined Model (Validation Data)")
# plt.xlabel('Predicted')
# plt.ylabel('Actual')
# plt.show()

# Calculate and print the false negative percentage for the combined model
false_negative_percentage_combined = calculate_false_negative(y_val, combined_val_predictions)
print(f"False Negative Percentage for Combined Model (Validation Data): {false_negative_percentage_combined:.2f}%")

# Calculate metrics for the combined model
accuracy_combined = accuracy_score(y_val, combined_val_predictions)
precision_combined = precision_score(y_val, combined_val_predictions, average='weighted')
recall_combined = recall_score(y_val, combined_val_predictions, average='weighted')
f1_combined = f1_score(y_val, combined_val_predictions, average='weighted')

# Print metrics for the combined model
print("Accuracy for Combined Model (Validation Data): ", accuracy_combined)
print("Precision for Combined Model (Validation Data): ", precision_combined)
print("Recall for Combined Model (Validation Data): ", recall_combined)
print("F1 Score for Combined Model (Validation Data): ", f1_combined)


False Negative Percentage for Combined Model (Validation Data): 1.01%
Accuracy for Combined Model (Validation Data):  0.975
Precision for Combined Model (Validation Data):  0.9751594227219332
Recall for Combined Model (Validation Data):  0.975
F1 Score for Combined Model (Validation Data):  0.9749166666666668


In [144]:
from collections import Counter
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score, confusion_matrix

# Define the function to calculate false negative percentage
def calculate_false_negative(y_true, y_pred):
    cm = confusion_matrix(y_true, y_pred)
    tn, fp, fn, tp = cm.ravel()
    return fn / (fn + tp) * 100

# Combine predictions using majority voting
combined_test_predictions = []
for p1, p2, p3, p4 in zip(test_predictions1, test_predictions2, test_predictions3, test_predictions4):
    votes = Counter([p1, p2, p3, p4])
    combined_test_predictions.append(votes.most_common(1)[0][0])

# Plot confusion matrix for the combined model
# cm_combined = confusion_matrix(y_test, combined_test_predictions)
# plt.figure(figsize=(8, 6))
# sns.heatmap(cm_combined, annot=True, fmt="d", cmap="Blues", xticklabels=np.unique(y_test), yticklabels=np.unique(y_test))
# plt.title("Confusion Matrix - Combined Model (Test Data)")
# plt.xlabel('Predicted')
# plt.ylabel('Actual')
# plt.show()

# Calculate and print the false negative percentage for the combined model
false_negative_percentage_combined = calculate_false_negative(y_test, combined_test_predictions)
print(f"False Negative Percentage for Combined Model (Test Data): {false_negative_percentage_combined:.2f}%")

# Calculate metrics for the combined model
accuracy_combined = accuracy_score(y_test, combined_test_predictions)
precision_combined = precision_score(y_test, combined_test_predictions, average='weighted')
recall_combined = recall_score(y_test, combined_test_predictions, average='weighted')
f1_combined = f1_score(y_test, combined_test_predictions, average='weighted')

# Print metrics for the combined model
print("Accuracy for Combined Model (Test Data): ", accuracy_combined)
print("Precision for Combined Model (Test Data): ", precision_combined)
print("Recall for Combined Model (Test Data): ", recall_combined)
print("F1 Score for Combined Model (Test Data): ", f1_combined)


False Negative Percentage for Combined Model (Test Data): 0.00%
Accuracy for Combined Model (Test Data):  0.99
Precision for Combined Model (Test Data):  0.9901834862385321
Recall for Combined Model (Test Data):  0.99
F1 Score for Combined Model (Test Data):  0.9899919484702093


False Negative Percentage for Combined Model (Test Data): 0.00%
Accuracy for Combined Model (Test Data):  0.99
Precision for Combined Model (Test Data):  0.9901834862385321
Recall for Combined Model (Test Data):  0.99
F1 Score for Combined Model (Test Data):  0.9899919484702093

# Output


In [145]:
import joblib

# Save the final trained stacking model
joblib.dump(sclf4, 'final_stacking_model3.pkl')

print("Final model saved successfully as 'final_stacking_model.pkl'!")


Final model saved successfully as 'final_stacking_model.pkl'!
